<a href="https://colab.research.google.com/github/yilmajung/LLM_POC_Study_2025_v2/blob/main/p2_finetune_llm_homosex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install transformers accelerate peft sentencepiece pandas pyarrow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os, json, math, random
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

In [5]:
# Set up paths
CS_CSV    = "/content/drive/MyDrive/LLM_POC_Study_2025_v2/gss_abt_cs_homosex.csv"       # cross-sectional long
PANEL_CSV = "/content/drive/MyDrive/LLM_POC_Study_2025_v2/gss_abt_panel_homosex.csv"    # panel long
OUT_DIR   = "/content/drive/MyDrive/LLM_POC_Study_2025_v2/outputs_gss_p_trial"          # where I write JSONLs & checkpoints
os.makedirs(OUT_DIR, exist_ok=True)

# LLM choice
BASE_MODEL_NAME = "meta-llama/llama-3.1-8b"  # or "mistralai/Mistral-7B-v0.3"

# Canonical bins (K=4)
ABORT4 = ["strong_anti", "anti", "pro", "strong_pro"]
TRUST3 = ["distrust", "depends", "trust"]
ENVIR3 = ["too_little", "about_right", "too_much"]
HOMOSEX4 = ["always_wrong", "almost_always_wrong", "sometimes_wrong", "not_wrong_at_all"]
ABORT2ID = {c:i for i,c in enumerate(ABORT4)}
TRUST2ID = {c:i for i,c in enumerate(TRUST3)}
ENVIR2ID = {c:i for i,c in enumerate(ENVIR3)}
HOMOSEX2ID = {c:i for i,c in enumerate(HOMOSEX4)}
K = len(HOMOSEX4)

YEARS_CS = list(range(2006, 2025, 2))  # 2006..2024 every 2 years

In [6]:
# Load and harmonize the data
# --- Cross-sectional ---
cs = pd.read_csv(CS_CSV)
# Expect: yearid, year, abortion_att4, generation, race, gender, edu_level, wtssps
# Map the attitude to canonical
# cs["att"] = cs["abortion_att4"].astype(str).str.strip()
# keep only canon categories, drop NAs
# cs_abortion = cs[cs["att"].isin(ABORT4)].copy()
# cs_abortion["wt"] = cs_abortion.get("wtssps", pd.Series([1.0]*len(cs_abortion)))  # default 1.0 if missing
# cs_trust = cs[cs["trust"].astype(str).str.strip().isin(TRUST3)].copy()
# cs_trust['wt'] = cs_trust.get("wtssps", pd.Series([1.0]*len(cs_trust)))  # default 1.0 if missing

# cs_envir = cs[cs["natenvir"].astype(str).str.strip().isin(ENVIR3)].copy()
# cs_envir['wt'] = cs_envir.get("wtssps", pd.Series([1.0]*len(cs_envir)))  # default 1.0 if missing

cs_homosex = cs[cs["homosex"].astype(str).str.strip().isin(HOMOSEX4)].copy()
cs_homosex['wt'] = cs_homosex.get("wtssps", pd.Series([1.0]*len(cs_homosex)))  # default 1.0 if missing

# --- Panel ---
pl = pd.read_csv(PANEL_CSV)
# pl["att"] = pl["abortion_att4"].astype(str).str.strip()
# pl_abortion = pl[pl["att"].isin(ABORT4)].copy()
# pl_trust = pl[pl["trust"].astype(str).str.strip().isin(TRUST3)].copy()
# pl_envir = pl[pl["natenvir"].astype(str).str.strip().isin(ENVIR3)].copy()
pl_homosex = pl[pl["homosex"].astype(str).str.strip().isin(HOMOSEX4)].copy()

# Define grouping keys
GROUP_COLS_4 = ["generation","gender","race","edu_level"]
GROUP_COLS_3 = ["generation","gender","race"]
GROUP_COLS_2 = ["generation","gender"]
GROUP_COLS_1 = ["generation"]

for df in (cs, pl):
    for c in GROUP_COLS_4:
        df[c] = df[c].astype(str).str.strip()

In [8]:
# Build cross-section margins p_cs[g,y] (weighted)
def group_key(row):
    return (row["generation"], row["gender"], row["race"], row['edu_level'])

def weighted_probs(vals, wts, cats=HOMOSEX4):
    # vals: list of category strings; wts: weights
    counts = {c:0.0 for c in cats}
    for v, w in zip(vals, wts):
        counts[v] += float(w)
    vec = np.array([counts[c] for c in cats], dtype=float)
    s = vec.sum()
    if s <= 0: return None
    return vec / s

In [7]:
# p_cs[(g,y)] -> np.array[K]
p_cs = {}
effN_cs = {}  # effective N for weighting samples in Task B
for y in YEARS_CS:
    sub = cs_homosex[cs_homosex["year"]==y]
    if sub.empty:
        continue
    for g_vals, df_g in sub.groupby(GROUP_COLS_4):
        p = weighted_probs(df_g["homosex"].tolist(), df_g["wt"].tolist(), HOMOSEX4)
        if p is None:
            continue
        p_cs[(g_vals, y)] = p
        effN_cs[(g_vals, y)] = float(df_g["wt"].sum())

# Build panel transitions C[(g,t,Δ)] from consecutive waves per yearid
# Build transitions per (g, t, Δ)
# C[(g,t,Δ)] -> KxK counts; Nfrom[(g,t,Δ)] -> row totals length K
from collections import defaultdict

def canon_index(cat):
    return HOMOSEX2ID.get(cat, None)

C = defaultdict(lambda: np.zeros((K,K), dtype=float))
Nfrom = defaultdict(lambda: np.zeros((K,), dtype=float))

# Weight=1.0
pl_homosex["w"] = 1.0

# Consecutive transitions per id
for (pid), df_id in pl_homosex.groupby("yearid"):
    df_id = df_id.sort_values("year")
    # collapse duplicates per year if any
    df_id = df_id.drop_duplicates(subset=["year"], keep="last")
    years = df_id["year"].values.tolist()
    homosexs  = df_id["homosex"].values.tolist()
    wgts  = df_id["w"].values.astype(float).tolist()
    gens  = df_id["generation"].values.tolist()
    gend  = df_id["gender"].values.tolist()
    race  = df_id["race"].values.tolist()
    edu = df_id["edu_level"].values.tolist()

    # require consistent group labels across waves for this id (common in panels)
    if not (len(set(gens))==1 and len(set(gend))==1 and len(set(race))==1 and len(set(edu))==1):
        # if you prefer, skip inconsistent cases
        continue

    g = (gens[0], gend[0], race[0], edu[0])
    for i in range(len(years)-1):
        t, t1 = int(years[i]), int(years[i+1])
        Δ = t1 - t
        if Δ not in (2,4):  # we only have 2-yr and 4-yr gaps here
            continue
        ai = canon_index(homosexs[i])
        aj = canon_index(homosexs[i+1])
        if ai is None or aj is None:
            continue
        w = float(wgts[i])  # or min(wgts[i], wgts[i+1])
        C[(g,t,Δ)][ai, aj] += w
        Nfrom[(g,t,Δ)][ai] += w


# JSONL builders: Task A (panel rows) and Task B (margins)
def smooth_row(row_counts, alpha):
    rc = np.array(row_counts, dtype=float) + alpha
    s = rc.sum()
    if s <= 0:
        return np.ones_like(rc)/len(rc)
    return rc / s

def build_taskA_rows(C, Nfrom, p_cs, out_jsonl, alpha_small=0.05, alpha_big=0.25, n_thresh=20, cap_weight=100.0):
    out = open(out_jsonl, "w", encoding="utf-8")
    n_rows = 0
    for (g,t,Δ), mat in C.items():
        nrow = Nfrom[(g,t,Δ)]
        for i in range(K):
            alpha = alpha_small if nrow[i] >= n_thresh else alpha_big
            tgt = smooth_row(mat[i,:], alpha=alpha).tolist()
            # Build prompt
            prompt = (
                "[Task: Predict transition row]\n"
                f"From: <Y{t}> → To: <Y{t+Δ}> <DT{Δ}>\n"
                f"Group: generation={g[0]}; gender={g[1]}; race={g[2]}; edu_level={g[3]}\n"
                f"From option: {HOMOSEX4[i]}\n"
                "Answer:\n"
            )
            rec = {
                "task": "row",
                "group": {"generation": g[0], "gender": g[1], "race": g[2], "edu_level": g[3]},
                "year_t": t,
                "year_t1": t+Δ,
                "dt": Δ,
                "from_bin": HOMOSEX4[i],
                "prompt_text": prompt,
                "to_dist": tgt,
                "weight": float(min(nrow[i], cap_weight))
            }
            # Attach margins if both available (for consistency loss later)
            if ((g,t) in p_cs) and ((g,t+Δ) in p_cs):
                rec["p_curr"] = p_cs[(g,t)].tolist()
                rec["p_next"] = p_cs[(g,t+Δ)].tolist()
            out.write(json.dumps(rec) + "\n")
            n_rows += 1
    out.close()
    return n_rows

def build_taskB_rows(p_cs, effN_cs, out_jsonl, lags=(2,4), cap_weight=500.0):
    out = open(out_jsonl, "w", encoding="utf-8")
    n_rows = 0
    # group by g
    by_g = {}
    for (g,y) in p_cs.keys():
        by_g.setdefault(g, []).append(y)
    for g, years in by_g.items():
        ys = sorted(years)
        for y in ys:
            # context from previous lags
            ctx = []
            for L in lags:
                yprev = y - L
                if (g, yprev) in p_cs:
                    ctx.append((yprev, p_cs[(g, yprev)]))
            if len(ctx) == 0:
                continue
            # Build prompt
            ctx_parts = " ".join([f"<Y{yy}>[{','.join(f'{x:.4f}' for x in p)}]" for (yy,p) in ctx])
            prompt = (
                "[Task: Forecast next-wave margin]\n"
                f"Group: generation={g[0]}; gender={g[1]}; race={g[2]}; edu_level={g[3]}\n"
                f"Context: {ctx_parts}\n"
                f"Predict: <Y{y}>\n"
                "Answer:\n"
            )
            w = float(min(effN_cs.get((g,y), 1.0), cap_weight))
            rec = {
                "task": "margin",
                "group": {"generation": g[0], "gender": g[1], "race": g[2], "edu_level": g[3]},
                "year": y,
                "prompt_text": prompt,
                "to_dist": p_cs[(g,y)].tolist(),
                "weight": w
            }
            out.write(json.dumps(rec) + "\n")
            n_rows += 1
    out.close()
    return n_rows

TASKA_JSONL = os.path.join(OUT_DIR, "taskA_panel_rows_homosex_g4.jsonl")
TASKB_JSONL = os.path.join(OUT_DIR, "taskB_margins_homosex_g4.jsonl")

na = build_taskA_rows(C, Nfrom, p_cs, TASKA_JSONL)
nb = build_taskB_rows(p_cs, effN_cs, TASKB_JSONL)
print(f"Wrote Task A rows: {na}")
print(f"Wrote Task B rows: {nb}")

Wrote Task A rows: 1476
Wrote Task B rows: 538


In [8]:
# Datasets and Collator (multitask)
class MTJsonlDataset(torch.utils.data.Dataset):
    def __init__(self, jsonl_paths: List[str], tokenizer, max_len=768):
        self.rows = []
        for p in jsonl_paths:
            with open(p, "r", encoding="utf-8") as f:
                self.rows.extend([json.loads(x) for x in f])
        random.shuffle(self.rows)
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.rows)

    def __getitem__(self, i):
        r = self.rows[i]
        enc = self.tok(r["prompt_text"], return_tensors="pt", truncation=True, max_length=self.max_len)
        out = {
            "task_type": r["task"],
            "input_ids": enc["input_ids"][0],
            "attention_mask": enc["attention_mask"][0],
            "to_dist": torch.tensor(r["to_dist"], dtype=torch.float),
            "weight": torch.tensor(float(r.get("weight", 1.0)), dtype=torch.float),
        }
        # add optional consistency fields
        if r["task"]=="row" and ("p_curr" in r) and ("p_next" in r):
            out["p_curr"] = torch.tensor(r["p_curr"], dtype=torch.float)
            out["p_next"] = torch.tensor(r["p_next"], dtype=torch.float)
            out["has_consistency"] = torch.tensor(1, dtype=torch.long)
        else:
            out["has_consistency"] = torch.tensor(0, dtype=torch.long)
        return out

# Model: LLM backbone + two small heads + last-K pooling
# Tokenizer & backbone
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def mt_collate(batch):
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    maxlen = max(x["input_ids"].shape[0] for x in batch)

    def pad(seq, pad_val, target_len):
        pad_n = target_len - seq.shape[0]
        if pad_n <= 0: return seq
        return torch.cat([seq, torch.full((pad_n,), pad_val, dtype=seq.dtype)])

    input_ids      = torch.stack([pad(x["input_ids"], pad_id, maxlen) for x in batch])
    attention_mask = torch.stack([pad(x["attention_mask"], 0, maxlen) for x in batch])
    to_dist        = torch.stack([x["to_dist"] for x in batch])
    weight         = torch.stack([x["weight"] for x in batch])
    has_cons       = torch.stack([x["has_consistency"] for x in batch])
    # p_curr/p_next if present; else zeros
    p_curr = torch.zeros(len(batch), K, dtype=torch.float)
    p_next = torch.zeros(len(batch), K, dtype=torch.float)
    for i, x in enumerate(batch):
        if x["has_consistency"]==1:
            p_curr[i] = x["p_curr"]
            p_next[i] = x["p_next"]

    task_types = [x["task_type"] for x in batch]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "to_dist": to_dist,
        "weight": weight,
        "has_consistency": has_cons,
        "p_curr": p_curr,
        "p_next": p_next,
        "task_types": task_types,
    }

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    output_hidden_states=True
)

# Add LoRA to attention/MLP projections
lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj"]
)
model = get_peft_model(base, lora_cfg)

# Two task heads
class TwoHead(nn.Module):
    def __init__(self, hidden_size, K):
        super().__init__()
        self.head_row    = nn.Linear(hidden_size, K)   # Task A
        self.head_margin = nn.Linear(hidden_size, K)   # Task B

    def forward(self, feats):
        return self.head_row(feats), self.head_margin(feats)

hidden_size = base.config.hidden_size
two_head = TwoHead(hidden_size, K).to(model.device)

# Simple pooled features: mean of last K tokens (tail window)
def pooled_features(outputs, attention_mask, tail=96):
    hs = outputs.hidden_states[-1]     # [B,T,H]
    B, T, H = hs.shape
    valid_lens = attention_mask.sum(dim=1)  # [B]
    feats = []
    for b in range(B):
        L = int(valid_lens[b].item())
        s = max(0, L - tail); e = L
        if e <= s: s, e = max(0, L-32), L
        feats.append(hs[b, s:e, :].mean(dim=0))
    feats = torch.stack(feats, dim=0)
    return feats

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [9]:
# Trainer with multitask losses (KL + optional consistency)
class MTTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        self.two_head = kwargs.pop("two_head")
        self.lambda_A = kwargs.pop("lambda_A", 1.0)
        self.lambda_B = kwargs.pop("lambda_B", 1.0)
        self.lambda_C = kwargs.pop("lambda_C", 0.5)
        super().__init__(*args, **kwargs)

    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: Optional[int] = None,  # <-- accept the kwarg
    ):
        input_ids      = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)
        to_dist        = inputs["to_dist"].to(model.device)     # [B,K]
        weight         = inputs["weight"].to(model.device)      # [B]
        has_cons       = inputs["has_consistency"].to(model.device)  # [B]
        p_curr         = inputs["p_curr"].to(model.device)      # [B,K]
        p_next         = inputs["p_next"].to(model.device)      # [B,K]
        task_types     = inputs["task_types"]                   # list[str]

        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        feats = pooled_features(out, attention_mask, tail=96).to(model.device)  # [B,H]

        logits_row, logits_margin = self.two_head(feats)   # [B,K], [B,K]
        p_row_hat    = F.softmax(logits_row, dim=1)
        p_margin_hat = F.softmax(logits_margin, dim=1)

        # Build masks by task
        is_row    = torch.tensor([t == "row"    for t in task_types], device=model.device, dtype=torch.bool)
        is_margin = torch.tensor([t == "margin" for t in task_types], device=model.device, dtype=torch.bool)

        eps = 1e-8
        def fwd_kl(p, q):
            p = p.clamp_min(eps); q = q.clamp_min(eps)
            return (p * (p.log() - q.log())).sum(dim=1)

        loss = torch.tensor(0.0, device=model.device)

        # Task A: panel rows
        if is_row.any():
            L_A = fwd_kl(to_dist[is_row], p_row_hat[is_row])
            loss = loss + self.lambda_A * (weight[is_row] * L_A).mean()

        # Task B: margins
        if is_margin.any():
            L_B = fwd_kl(to_dist[is_margin], p_margin_hat[is_margin])
            loss = loss + self.lambda_B * (weight[is_margin] * L_B).mean()

        # Consistency (disabled unless you wire full-row assembly):
        # cons_mask = is_row & (has_cons==1)
        # if cons_mask.any() and self.lambda_C > 0:
        #     ...

        if return_outputs:
            return loss, {"logits_row": logits_row, "logits_margin": logits_margin, "labels": to_dist}
        else:
            return loss



In [10]:
# Training setup
tokenizer.padding_side = "left"  # (helps with some backbones; optional)

train_ds = MTJsonlDataset([TASKA_JSONL, TASKB_JSONL], tokenizer, max_len=768)
# For a quick start you can split train/val here:
val_frac = 0.1
n_val = int(len(train_ds) * val_frac)
indices = list(range(len(train_ds)))
random.seed(0); random.shuffle(indices)
val_idx  = set(indices[:n_val])
train_idx= set(indices[n_val:])

class SubsetDS(torch.utils.data.Dataset):
    def __init__(self, base, keep_idx):
        self.base = base
        self.keep = sorted(list(keep_idx))
    def __len__(self): return len(self.keep)
    def __getitem__(self, i): return self.base[self.keep[i]]

ds_train = SubsetDS(train_ds, train_idx)
ds_val   = SubsetDS(train_ds, val_idx)

args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "ckpt"),
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=20,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    bf16=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    warmup_steps=300,
    weight_decay=0.0,
    report_to="none",
    remove_unused_columns=False,  # keep our custom fields
)

trainer = MTTrainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    data_collator=mt_collate,
    two_head=two_head,
    lambda_A=1.0,
    lambda_B=1.0,
    lambda_C=0.0,   # keep 0.0 for now; we’ll add strict consistency later
)

trainer.train()

# Save adapter + heads
save_dir = os.path.join(OUT_DIR, "final_multitask_homosex_g4")
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
torch.save(two_head.state_dict(), os.path.join(save_dir, "two_head_homosex_g4.pt"))
print("Saved to:", save_dir)

/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


Step,Training Loss,Validation Loss
200,4.423900,No log
400,3.427700,No log
600,2.472800,No log
800,2.268400,No log
1000,2.790300,No log
1200,2.795600,No log
1400,2.194100,No log
1600,2.245800,No log
1800,1.842200,No log
2000,1.552200,No log


/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', arg

Saved to: /content/drive/MyDrive/LLM_POC_Study_2025_v2/outputs_gss_p_trial/final_multitask_homosex_g4


### HOMOSEX, Group 3

In [9]:
# p_cs[(g,y)] -> np.array[K]
p_cs = {}
effN_cs = {}  # effective N for weighting samples in Task B
for y in YEARS_CS:
    sub = cs_homosex[cs_homosex["year"]==y]
    if sub.empty:
        continue
    for g_vals, df_g in sub.groupby(GROUP_COLS_3):
        p = weighted_probs(df_g["homosex"].tolist(), df_g["wt"].tolist(), HOMOSEX4)
        if p is None:
            continue
        p_cs[(g_vals, y)] = p
        effN_cs[(g_vals, y)] = float(df_g["wt"].sum())

# Build panel transitions C[(g,t,Δ)] from consecutive waves per yearid
# Build transitions per (g, t, Δ)
# C[(g,t,Δ)] -> KxK counts; Nfrom[(g,t,Δ)] -> row totals length K
from collections import defaultdict

def canon_index(cat):
    return HOMOSEX2ID.get(cat, None)

C = defaultdict(lambda: np.zeros((K,K), dtype=float))
Nfrom = defaultdict(lambda: np.zeros((K,), dtype=float))

# Weight=1.0
pl_homosex["w"] = 1.0

# Consecutive transitions per id
for (pid), df_id in pl_homosex.groupby("yearid"):
    df_id = df_id.sort_values("year")
    # collapse duplicates per year if any
    df_id = df_id.drop_duplicates(subset=["year"], keep="last")
    years = df_id["year"].values.tolist()
    homosexs  = df_id["homosex"].values.tolist()
    wgts  = df_id["w"].values.astype(float).tolist()
    gens  = df_id["generation"].values.tolist()
    gend  = df_id["gender"].values.tolist()
    race  = df_id["race"].values.tolist()
    #edu = df_id["edu_level"].values.tolist()

    # require consistent group labels across waves for this id (common in panels)
    if not (len(set(gens))==1 and len(set(gend))==1 and len(set(race))==1):
        # if you prefer, skip inconsistent cases
        continue

    g = (gens[0], gend[0], race[0])
    for i in range(len(years)-1):
        t, t1 = int(years[i]), int(years[i+1])
        Δ = t1 - t
        if Δ not in (2,4):  # we only have 2-yr and 4-yr gaps here
            continue
        ai = canon_index(homosexs[i])
        aj = canon_index(homosexs[i+1])
        if ai is None or aj is None:
            continue
        w = float(wgts[i])  # or min(wgts[i], wgts[i+1])
        C[(g,t,Δ)][ai, aj] += w
        Nfrom[(g,t,Δ)][ai] += w


# JSONL builders: Task A (panel rows) and Task B (margins)
def smooth_row(row_counts, alpha):
    rc = np.array(row_counts, dtype=float) + alpha
    s = rc.sum()
    if s <= 0:
        return np.ones_like(rc)/len(rc)
    return rc / s

def build_taskA_rows(C, Nfrom, p_cs, out_jsonl, alpha_small=0.05, alpha_big=0.25, n_thresh=20, cap_weight=100.0):
    out = open(out_jsonl, "w", encoding="utf-8")
    n_rows = 0
    for (g,t,Δ), mat in C.items():
        nrow = Nfrom[(g,t,Δ)]
        for i in range(K):
            alpha = alpha_small if nrow[i] >= n_thresh else alpha_big
            tgt = smooth_row(mat[i,:], alpha=alpha).tolist()
            # Build prompt
            prompt = (
                "[Task: Predict transition row]\n"
                f"From: <Y{t}> → To: <Y{t+Δ}> <DT{Δ}>\n"
                f"Group: generation={g[0]}; gender={g[1]}; race={g[2]}\n"
                f"From option: {HOMOSEX4[i]}\n"
                "Answer:\n"
            )
            rec = {
                "task": "row",
                "group": {"generation": g[0], "gender": g[1], "race": g[2]},
                "year_t": t,
                "year_t1": t+Δ,
                "dt": Δ,
                "from_bin": HOMOSEX4[i],
                "prompt_text": prompt,
                "to_dist": tgt,
                "weight": float(min(nrow[i], cap_weight))
            }
            # Attach margins if both available (for consistency loss later)
            if ((g,t) in p_cs) and ((g,t+Δ) in p_cs):
                rec["p_curr"] = p_cs[(g,t)].tolist()
                rec["p_next"] = p_cs[(g,t+Δ)].tolist()
            out.write(json.dumps(rec) + "\n")
            n_rows += 1
    out.close()
    return n_rows

def build_taskB_rows(p_cs, effN_cs, out_jsonl, lags=(2,4), cap_weight=500.0):
    out = open(out_jsonl, "w", encoding="utf-8")
    n_rows = 0
    # group by g
    by_g = {}
    for (g,y) in p_cs.keys():
        by_g.setdefault(g, []).append(y)
    for g, years in by_g.items():
        ys = sorted(years)
        for y in ys:
            # context from previous lags
            ctx = []
            for L in lags:
                yprev = y - L
                if (g, yprev) in p_cs:
                    ctx.append((yprev, p_cs[(g, yprev)]))
            if len(ctx) == 0:
                continue
            # Build prompt
            ctx_parts = " ".join([f"<Y{yy}>[{','.join(f'{x:.4f}' for x in p)}]" for (yy,p) in ctx])
            prompt = (
                "[Task: Forecast next-wave margin]\n"
                f"Group: generation={g[0]}; gender={g[1]}; race={g[2]}\n"
                f"Context: {ctx_parts}\n"
                f"Predict: <Y{y}>\n"
                "Answer:\n"
            )
            w = float(min(effN_cs.get((g,y), 1.0), cap_weight))
            rec = {
                "task": "margin",
                "group": {"generation": g[0], "gender": g[1], "race": g[2]},
                "year": y,
                "prompt_text": prompt,
                "to_dist": p_cs[(g,y)].tolist(),
                "weight": w
            }
            out.write(json.dumps(rec) + "\n")
            n_rows += 1
    out.close()
    return n_rows

TASKA_JSONL = os.path.join(OUT_DIR, "taskA_panel_rows_homosex_g3.jsonl")
TASKB_JSONL = os.path.join(OUT_DIR, "taskB_margins_homosex_g3.jsonl")

na = build_taskA_rows(C, Nfrom, p_cs, TASKA_JSONL)
nb = build_taskB_rows(p_cs, effN_cs, TASKB_JSONL)
print(f"Wrote Task A rows: {na}")
print(f"Wrote Task B rows: {nb}")

Wrote Task A rows: 676
Wrote Task B rows: 210


In [10]:
# Datasets and Collator (multitask)
class MTJsonlDataset(torch.utils.data.Dataset):
    def __init__(self, jsonl_paths: List[str], tokenizer, max_len=768):
        self.rows = []
        for p in jsonl_paths:
            with open(p, "r", encoding="utf-8") as f:
                self.rows.extend([json.loads(x) for x in f])
        random.shuffle(self.rows)
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self): return len(self.rows)

    def __getitem__(self, i):
        r = self.rows[i]
        enc = self.tok(r["prompt_text"], return_tensors="pt", truncation=True, max_length=self.max_len)
        out = {
            "task_type": r["task"],
            "input_ids": enc["input_ids"][0],
            "attention_mask": enc["attention_mask"][0],
            "to_dist": torch.tensor(r["to_dist"], dtype=torch.float),
            "weight": torch.tensor(float(r.get("weight", 1.0)), dtype=torch.float),
        }
        # add optional consistency fields
        if r["task"]=="row" and ("p_curr" in r) and ("p_next" in r):
            out["p_curr"] = torch.tensor(r["p_curr"], dtype=torch.float)
            out["p_next"] = torch.tensor(r["p_next"], dtype=torch.float)
            out["has_consistency"] = torch.tensor(1, dtype=torch.long)
        else:
            out["has_consistency"] = torch.tensor(0, dtype=torch.long)
        return out

# Model: LLM backbone + two small heads + last-K pooling
# Tokenizer & backbone
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def mt_collate(batch):
    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id
    maxlen = max(x["input_ids"].shape[0] for x in batch)

    def pad(seq, pad_val, target_len):
        pad_n = target_len - seq.shape[0]
        if pad_n <= 0: return seq
        return torch.cat([seq, torch.full((pad_n,), pad_val, dtype=seq.dtype)])

    input_ids      = torch.stack([pad(x["input_ids"], pad_id, maxlen) for x in batch])
    attention_mask = torch.stack([pad(x["attention_mask"], 0, maxlen) for x in batch])
    to_dist        = torch.stack([x["to_dist"] for x in batch])
    weight         = torch.stack([x["weight"] for x in batch])
    has_cons       = torch.stack([x["has_consistency"] for x in batch])
    # p_curr/p_next if present; else zeros
    p_curr = torch.zeros(len(batch), K, dtype=torch.float)
    p_next = torch.zeros(len(batch), K, dtype=torch.float)
    for i, x in enumerate(batch):
        if x["has_consistency"]==1:
            p_curr[i] = x["p_curr"]
            p_next[i] = x["p_next"]

    task_types = [x["task_type"] for x in batch]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "to_dist": to_dist,
        "weight": weight,
        "has_consistency": has_cons,
        "p_curr": p_curr,
        "p_next": p_next,
        "task_types": task_types,
    }

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    output_hidden_states=True
)

# Add LoRA to attention/MLP projections
lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj"]
)
model = get_peft_model(base, lora_cfg)

# Two task heads
class TwoHead(nn.Module):
    def __init__(self, hidden_size, K):
        super().__init__()
        self.head_row    = nn.Linear(hidden_size, K)   # Task A
        self.head_margin = nn.Linear(hidden_size, K)   # Task B

    def forward(self, feats):
        return self.head_row(feats), self.head_margin(feats)

hidden_size = base.config.hidden_size
two_head = TwoHead(hidden_size, K).to(model.device)

# Simple pooled features: mean of last K tokens (tail window)
def pooled_features(outputs, attention_mask, tail=96):
    hs = outputs.hidden_states[-1]     # [B,T,H]
    B, T, H = hs.shape
    valid_lens = attention_mask.sum(dim=1)  # [B]
    feats = []
    for b in range(B):
        L = int(valid_lens[b].item())
        s = max(0, L - tail); e = L
        if e <= s: s, e = max(0, L-32), L
        feats.append(hs[b, s:e, :].mean(dim=0))
    feats = torch.stack(feats, dim=0)
    return feats

`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Trainer with multitask losses (KL + optional consistency)
class MTTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        self.two_head = kwargs.pop("two_head")
        self.lambda_A = kwargs.pop("lambda_A", 1.0)
        self.lambda_B = kwargs.pop("lambda_B", 1.0)
        self.lambda_C = kwargs.pop("lambda_C", 0.5)
        super().__init__(*args, **kwargs)

    def compute_loss(
        self,
        model,
        inputs,
        return_outputs: bool = False,
        num_items_in_batch: Optional[int] = None,  # <-- accept the kwarg
    ):
        input_ids      = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)
        to_dist        = inputs["to_dist"].to(model.device)     # [B,K]
        weight         = inputs["weight"].to(model.device)      # [B]
        has_cons       = inputs["has_consistency"].to(model.device)  # [B]
        p_curr         = inputs["p_curr"].to(model.device)      # [B,K]
        p_next         = inputs["p_next"].to(model.device)      # [B,K]
        task_types     = inputs["task_types"]                   # list[str]

        with torch.cuda.amp.autocast(dtype=torch.bfloat16):
            out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        feats = pooled_features(out, attention_mask, tail=96).to(model.device)  # [B,H]

        logits_row, logits_margin = self.two_head(feats)   # [B,K], [B,K]
        p_row_hat    = F.softmax(logits_row, dim=1)
        p_margin_hat = F.softmax(logits_margin, dim=1)

        # Build masks by task
        is_row    = torch.tensor([t == "row"    for t in task_types], device=model.device, dtype=torch.bool)
        is_margin = torch.tensor([t == "margin" for t in task_types], device=model.device, dtype=torch.bool)

        eps = 1e-8
        def fwd_kl(p, q):
            p = p.clamp_min(eps); q = q.clamp_min(eps)
            return (p * (p.log() - q.log())).sum(dim=1)

        loss = torch.tensor(0.0, device=model.device)

        # Task A: panel rows
        if is_row.any():
            L_A = fwd_kl(to_dist[is_row], p_row_hat[is_row])
            loss = loss + self.lambda_A * (weight[is_row] * L_A).mean()

        # Task B: margins
        if is_margin.any():
            L_B = fwd_kl(to_dist[is_margin], p_margin_hat[is_margin])
            loss = loss + self.lambda_B * (weight[is_margin] * L_B).mean()

        # Consistency (disabled unless you wire full-row assembly):
        # cons_mask = is_row & (has_cons==1)
        # if cons_mask.any() and self.lambda_C > 0:
        #     ...

        if return_outputs:
            return loss, {"logits_row": logits_row, "logits_margin": logits_margin, "labels": to_dist}
        else:
            return loss



In [12]:
# Training setup
tokenizer.padding_side = "left"  # (helps with some backbones; optional)

train_ds = MTJsonlDataset([TASKA_JSONL, TASKB_JSONL], tokenizer, max_len=768)
# For a quick start you can split train/val here:
val_frac = 0.1
n_val = int(len(train_ds) * val_frac)
indices = list(range(len(train_ds)))
random.seed(0); random.shuffle(indices)
val_idx  = set(indices[:n_val])
train_idx= set(indices[n_val:])

class SubsetDS(torch.utils.data.Dataset):
    def __init__(self, base, keep_idx):
        self.base = base
        self.keep = sorted(list(keep_idx))
    def __len__(self): return len(self.keep)
    def __getitem__(self, i): return self.base[self.keep[i]]

ds_train = SubsetDS(train_ds, train_idx)
ds_val   = SubsetDS(train_ds, val_idx)

args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "ckpt"),
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=20,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    bf16=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    warmup_steps=300,
    weight_decay=0.0,
    report_to="none",
    remove_unused_columns=False,  # keep our custom fields
)

trainer = MTTrainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    data_collator=mt_collate,
    two_head=two_head,
    lambda_A=1.0,
    lambda_B=1.0,
    lambda_C=0.0,   # keep 0.0 for now; we’ll add strict consistency later
)

trainer.train()

# Save adapter + heads
save_dir = os.path.join(OUT_DIR, "final_multitask_homosex_g3")
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
torch.save(two_head.state_dict(), os.path.join(save_dir, "two_head_homosex_g3.pt"))
print("Saved to:", save_dir)

/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):


Step,Training Loss,Validation Loss
200,6.270700,No log
400,5.043900,No log
600,3.520400,No log
800,3.587700,No log
1000,2.430300,No log
1200,1.837600,No log
1400,1.536500,No log
1600,0.888700,No log
1800,0.627500,No log
2000,0.598500,No log


/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.bfloat16):
/tmp/ipython-input-4224191036.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', arg

Saved to: /content/drive/MyDrive/LLM_POC_Study_2025_v2/outputs_gss_p_trial/final_multitask_homosex_g3
